In [139]:
from qiskit import *
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor

In [140]:
class QBitGen:
    def __init__(self):
        self.__random_gen = ""
        self.__qrn_circ = None
        self.__failed_results = False
        self.__provider = None

    # Setup the circuit 
    def init_circ(self):
        qcirc = QuantumCircuit(1, 1)

        # prepare |+> state
        qcirc.h(0)
        qcirc.barrier()

        # Perform measurements in z-basis
        qcirc.measure(0, 0)

        self.__qrn_circ = qcirc
        
            
    # Return the random, generated sequence, or "" if error
    def get_random(self)->str:
        if self.__random_gen == "" or self.__failed_results == True:
            print("Could not fetch random or out of range, try to generate it again")
            return ""
        return self.__random_gen
    
    # Run the circuit in simulator
    def simulate_circ(self, num_shots=4096):        
        if(self.__qrn_circ == None):
            print("Initialize the circuit first")
            return
        
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(self.__qrn_circ, shots = num_shots, memory=True)
        result = aer_sim.run(qobj).result().get_memory()
        
        # Calculate X
        random_gen = 0
        for i in range(len(result)):
            random_gen = random_gen + int(result[i])/(pow(2,i+1))
            
        if random_gen == 0:
            self.__failed_results = True
            return

        self.__failed_results = False
        self.__random_gen = random_gen

    # Load the IBMQ account
    def load_provider(self):
        try:
            IBMQ.load_account()
            self.__provider = IBMQ.get_provider(hub='ibm-q')
        except IBMQProviderError:
            print("Could not fetch providers")


    # Run on the real hardware
    def evaluate_circ(self, num_shots=100):
        if self.__qrn_circ == None:
            print("Initialize the circuit first")
            return
        
        if self.__provider == None:
            print("Load the available IBMQ providers first")
            return
        
        assert(num_shots > 0)

        # Run the job on the least busy backend and store the results
        backend = least_busy(self.__provider.backends(filters=lambda b: b.configuration().n_qubits == 5 and
                                   not b.configuration().simulator and b.status().operational==True))

        job = backend.run(transpile(self.__qrn_circ, backend, optimization_level=3), 
                          memory=True, meas_level=2, shots=num_shots)
        job_monitor(job)
        result = job.result().get_memory()
        
        # Calculate X
        random_gen = 0
        for i in range(len(result)):
            random_gen = random_gen + int(result[i])/(pow(2,i+1))
            
        if random_gen == 0:
            self.__failed_results = True
            return

        self.__failed_results = False
        self.__random_gen = random_gen

In [141]:
# Can first verify using the simulator:

#qbg = QBitGen()
#qbg.init_circ()
#qbg.simulate_circ()
#print(qbg.get_random())

In [142]:
# Then run on a run quantum computer:

qbg = QBitGen()
qbg.init_circ()
qbg.load_provider()
qbg.evaluate_circ(20)

ibmqfactory.load_account:WARNING:2022-04-22 01:12:49,330: Credentials are already in use. The existing account in the session will be replaced.


Job Status: job is queued (27)    

KeyboardInterrupt: 

In [ ]:
print(qbg.get_random())
print(len(qbg.get_random()))